# Get The Coins Game

## 1. Needed Libraries

In [ ]:
# External libraries

import cv2
from cv2 import aruco
import numpy as np

from time import sleep
from objloader_simple import OBJ  # Custom OBJ file loader

import math
import open3d as o3d

from typing import List
from PIL import Image
import threading
import multiprocessing
import time

from multiprocessing.managers import BaseManager

import pygame  # Used for rendering the game visuals

# Own libraries

from my_logger import MyLogger

## 2. Create logger instance

In [ ]:
logger = MyLogger(level='DEBUG', show_timestamp=False)

## 3. Global Variables Definition

In [ ]:
# Screen resolution
SCREEN_WIDTH = 1980
SCREEN_HEIGHT = 1080

# Dimensions for the base game window
WINDOW_BASE_WIDTH = 900
WINDOW_BASE_HEIGHT = 675

## 4. Definition and Implementation of the Coin Class

### 4.1. CoinBase Class for Storing the Shared Variables

In [ ]:
class CoinBase:
    """
    Represents a virtual coin in the world, with position, rotation, visibility,
    and pixel info.

    :cvar COIN_SIZE: Diameter in centimeters.
    :type COIN_SIZE: float
    :cvar ROTATION_STEP: Rotation step per frame in radians.
    :type ROTATION_STEP: float
    """
    # Coin constants
    COIN_SIZE: float = 3.0
    ROTATION_STEP: float = 0.01

    def __init__(self, position: np.array) -> None:
        """
        Initializes the coin at a given 3D world position.

        :param position: A numpy array of shape (3,) representing the coin's
                         initial 3D position (x, y, z) in centimeters.
        :type position: np.array

        :rtype: None
        """
        # World positions (current and target)
        self._world_pose: np.array = position
        self._new_world_pose: np.array = position

        # World rotation
        self._world_rotation_z: float = 0.0

        # Coin visibility flag
        self._coin_visibility: bool = True

        # Last time the coin was displayed
        self._last_apparition_time: float = 0.0

    def set_pose(self, new_pose: np.array) -> None:
        """
        Sets a new 3D world position for the coin (in cm).

        :param new_pose: A numpy array representing the new 3D world position
                         (x, y, z) of the coin in centimeters.
        :type new_pose: np.array

        :rtype: None
        """
        self._world_pose = new_pose

    def get_pose(self) -> np.array:
        """
        Returns the coin's current 3D position in world coordinates (in cm).

        :return: A numpy array representing the coin's current 3D position
                 (x, y, z) in centimeters.
        :rtype: np.array
        """
        return self._world_pose

    def increment_rotation(self) -> None:
        """
        Increases the Z rotation by a small step (in radians).

        :rtype: None
        """
        self._world_rotation_z += self.ROTATION_STEP

    def get_rotation(self) -> float:
        """
        Returns the coin's current rotation around the Z axis (in radians).

        :return: The current rotation of the coin around the Z axis in radians.
        :rtype: float
        """
        return self._world_rotation_z

    def set_new_pose(self, new_pose: np.array) -> None:
        """
        Stores a new target 3D position for the coin (in cm).

        :param new_pose: A numpy array representing the new target 3D position
                         (x, y, z) in centimeters.
        :type new_pose: np.array

        :rtype: None
        """
        self._new_world_pose = new_pose

    def get_new_pose(self) -> np.array:
        """
        Returns the stored new 3D position (in cm).

        :return: A numpy array representing the stored target 3D position
                 (x, y, z) in centimeters.
        :rtype: np.array
        """
        return self._new_world_pose

    def set_visibility(self, visibility: bool) -> None:
        """
        Updates the coin's visibility flag.

        :param visibility: A boolean indicating whether the coin should be
                           visible or not.
        :type visibility: bool

        :rtype: None
        """
        self._coin_visibility = visibility

    def is_visible(self) -> bool:
        """
        Returns whether the coin is currently visible.

        :return: A boolean indicating if the coin is visible ('True') or not
                 ('False').
        :rtype: bool
        """
        return self._coin_visibility

    def set_last_apparition_time(self, time: float) -> None:
        """
        Sets the timestamp of the coin's last appearance.

        :param time: A float representing the last appearance timestamp of the
                     coin.
        :type time: float

        :rtype: None
        """
        self._last_apparition_time = time

    def get_last_apparition_time(self) -> float:
        """
        Returns the last recorded time the coin appeared.

        :return: The timestamp (in seconds) of the coin's last appearance.
        :rtype: float
        """
        return self._last_apparition_time

    def get_coin_radius(self) -> float:
        """
        Returns the coin's radius in cm (half of its size).

        :return: The radius of the coin in centimeters.
        :rtype: float
        """
        return self.COIN_SIZE / 2

    @staticmethod
    def random_coin_position(max_x: float, max_y: float) -> np.array:
        """
        Generates a random 3D position for the coin within the given bounds.

        :param max_x: The maximum x-coordinate value for the coin's position.
        :type max_x: float
        :param max_y: The maximum y-coordinate value for the coin's position.
        :type max_y: float

        :return: A numpy array representing the randomly generated 3D position
                 (x, y, z) of the coin in centimeters.
        :rtype: np.array
        """
        x = np.random.uniform(
            0.0 + CoinBase.COIN_SIZE, max_x - CoinBase.COIN_SIZE)

        y = np.random.uniform(
            0 + CoinBase.COIN_SIZE, max_y - CoinBase.COIN_SIZE)

        position = np.array([x, y, CoinBase.COIN_SIZE / 2])

        return position

### 4.2. Coin Class Implementation

In [ ]:
class Coin:
    """
    Represents the 3D visual and logic model of a coin.
    This class handles the geometry and the visual state of a coin in a 3D
    virtual scene.
    
    :ivar: shared_variables: Shared state with position, rotation, etc.
    :type: shared_variables: CoinBase
    :ivar: augmented_reality_obj: Augmented Reality overlay model.
    :type: augmented_reality_obj: OBJ
    :ivar: is_added_in_visualizer: To check if is added in visualizer.
    :type: bool
    """

    def __init__(self, shared_variables: CoinBase) -> None:
        """
        Initializes the coin mesh and sets its initial pose.

        :param shared_variables: An instance of the CoinBase class, which holds
                                 shared state information such as the position,
                                 rotation, and other state variables for the coin.

        :rtype: None
        """
        # Shared state with position, rotation, etc.
        self.shared_variables: CoinBase = shared_variables

        # Load the base 3D coin model
        self._triangle_mesh = o3d.io.read_triangle_mesh(
            "./media/models/virtual_coin.obj")

        # Scale the model so it has 3 cm height
        scale_factor = (CoinBase.COIN_SIZE / max(
            self._triangle_mesh.get_max_bound() -
            self._triangle_mesh.get_min_bound()))

        self._triangle_mesh.scale(
            scale_factor, center=self._triangle_mesh.get_center())

        # Rotate model so it's upright
        R = self._triangle_mesh.get_rotation_matrix_from_xyz((np.pi / 2, 0, 0))
        self._triangle_mesh.rotate(R, center=self._triangle_mesh.get_center())

        # Move model to origin
        self._triangle_mesh.translate(-(self._triangle_mesh.get_center()))

        # Set initial world position
        self._triangle_mesh.translate(self.shared_variables.get_pose())

        # Setup mesh properties
        self._triangle_mesh.compute_vertex_normals()
        self._triangle_mesh.paint_uniform_color([1.0, 0.83, 0.46])  # Yellow

        # Load Augmented Reality overlay model
        self.augmented_reality_obj = OBJ(
            './media/models/augmented_reality_coin.obj', swapyz=True)
        
        self.is_added_in_visualizer = False

    def update_pose(self) -> None:
        """
        Updates the coin's position to its new world pose.

        This method resets the coin's position to the origin, then updates
        the position based on the coin's current target pose.

        :rtype: None
        """
        # Reset to origin
        self._triangle_mesh.translate(
            -(self.shared_variables.get_pose()), relative=True)

        # Apply new position
        self.shared_variables.set_pose(
            self.shared_variables.get_new_pose())

        self._triangle_mesh.translate(
            self.shared_variables.get_pose(), relative=True)

    def update_rotation(self) -> None:
        """
        Rotates the coin by a small step around the Z axis.

        This method applies a rotation to the coin by a fixed step size,
        updating the coin's rotational state.

        :rtype: None
        """
        self.shared_variables.increment_rotation()

        R = self._triangle_mesh.get_rotation_matrix_from_xyz(
            (0, 0, CoinBase.ROTATION_STEP))

        self._triangle_mesh.rotate(R, center=self._triangle_mesh.get_center())

    def get_triangle_mesh(self) -> o3d.geometry.TriangleMesh:
        """
        Returns the Open3D mesh of the coin.

        :return: The Open3D triangle mesh representing the 3D geometry of the
                 coin.
        :rtype: o3d.geometry.TriangleMesh
        """
        return self._triangle_mesh

    def update_virtual_pose(self, new_pose: np.array) -> None:
        """
        Sets a new target pose for the coin (in cm).

        :param new_pose: A numpy array representing the new target position of
                         the coin in centimeters (x, y, z).
        :type: new_pose: np.array

        :rtype: None
        """
        self.shared_variables.set_new_pose(new_pose)

## 5. Definition and Implementation of the Pentagon Class

### 5.1. PentagonBase Class Storing the Shared Variables

In [ ]:
class PentagonBase:
    """
    Logical representation of a pentagon object with pose and screen position 
    tracking.

    :cvar PENTAGON_HEIGHT: Height in centimeters.
    :type PENTAGON_HEIGHT: float
    :cvar PENTAGON_RADIUS: Radius in centimeters.
    :type PENTAGON_RADIUS: float
    """
    # Physical dimensions (in cm)
    PENTAGON_HEIGHT: float = 1.0
    PENTAGON_RADIUS: float = 1.3

    def __init__(self) -> None:
        """
        Initializes default world and pixel positions.
        
        :rtype: None
        """
        # World positions (current and target)
        self._world_pose = np.array([0.0, 0.0, 0.0])
        self._new_world_pose = np.array([0.0, 0.0, 0.0])

    def set_pose(self, new_pose: np.array) -> None:
        """
        Sets the current world position of the pentagon.

        :param new_pose: A numpy array representing the new world position
                         (x, y, z) of the pentagon in centimeters.
        :type: new_pose: np.array

        :rtype: None
        """
        self._world_pose = new_pose

    def get_pose(self) -> np.array:
        """
        Returns the current world position of the pentagon.

        :return: The current world position of the pentagon in centimeters
                 (x, y, z).
        :rtype: np.array
        """
        return self._world_pose

    def set_new_pose(self, new_pose: np.array) -> None:
        """
        Sets a new target world position for the pentagon.

        :param: new_pose: A numpy array representing the target world position
                          of the pentagon in centimeters (x, y, z).
        :type: new_pose: np.array

        :rtype: None
        """
        self._new_world_pose = new_pose

    def get_new_pose(self) -> np.array:
        """
        Returns the target world position of the pentagon.

        :return: The target world position of the pentagon in centimeters
                 (x, y, z).
        :rtype: np.array
        """
        return self._new_world_pose

    def get_pentagon_radius(self):
        """
        Returns the pentagon's radius.

        :return: Radius of the pentagon in centimeters.
        :rtype: float
        """
        return self.PENTAGON_RADIUS

### 5.2. Pentagon Class Implementation

In [ ]:
class Pentagon:
    """
    Visual and geometric representation of a 3D pentagon prism.

    :ivar: shared_variables: Shared state with position, etc.
    :type: shared_variables: PentagonBase
    """

    def __init__(self, shared_variables: PentagonBase) -> None:
        """
        Initializes the pentagon mesh and related geometry.
         
        :param shared_variables: An instance of the class that holds shared
                                 variables related to the pentagon's world
                                 position and other state variables.
        :type: shared_variables: PentagonBase
        
        :rtype: None
        """
        self.shared_variables = shared_variables

        # Create the base vertices of the pentagon
        angle = np.linspace(0, 2 * np.pi, 6)[:-1]
        x = PentagonBase.PENTAGON_RADIUS * np.cos(angle)
        y = PentagonBase.PENTAGON_RADIUS * np.sin(angle)

        # Create the base face (z = 0)
        z = np.array([0, 0, 0, 0, 0])
        vertices = np.vstack((x, y, z)).T

        # Create the top face
        z_top = np.array([PentagonBase.PENTAGON_HEIGHT] * 5) # Pentagon height
        vertices_top = np.vstack((x, y, z_top)).T

        # Combine base and top vertices
        vertices = np.vstack((vertices, vertices_top))

        # Define triangular faces for sides and caps
        faces = []
        for i in range(5):
            # Side faces (2 triangles per side)
            faces.append([i, (i + 1) % 5, 5 + (i + 1) % 5])
            faces.append([i, 5 + (i + 1) % 5, 5 + i])

            # Bottom face (fan from vertex 0)
        faces.append([0, 1, 2])
        faces.append([0, 2, 3])
        faces.append([0, 3, 4])
        faces.append([0, 4, 1])

        # Top face (fan from vertex 5)
        faces.append([5, 6, 7])
        faces.append([5, 7, 8])
        faces.append([5, 8, 9])
        faces.append([5, 9, 6])

        # Create and store triangle mesh
        self._triangle_mesh = o3d.geometry.TriangleMesh()
        self._triangle_mesh.vertices = o3d.utility.Vector3dVector(vertices)
        self._triangle_mesh.triangles = o3d.utility.Vector3iVector(faces)

        # Improve shading
        self._triangle_mesh.compute_vertex_normals()

        # Set color
        self._triangle_mesh.paint_uniform_color([0.67, 0.98, 0.56])

    def get_triangle_mesh(self) -> o3d.geometry.TriangleMesh:
        """
        Returns the Open3D triangle mesh object.

        :return: The Open3D triangle mesh representing the pentagon.
        :rtype: o3d.geometry.TriangleMesh
        """
        return self._triangle_mesh

    def update_virtual_pose(self, new_pose: np.array) -> None:
        """
        Updates the virtual position of the pentagon (meters to cm).

        :param new_pose: A numpy array or list representing the new position
                         of the pentagon in meters. The values will be converted
                         to centimeters when updated.
        :type new_pose: np.array

        :rtype: None
        """
        self.shared_variables.set_new_pose(100 * new_pose)

## 6. Definition and Implementation of the VirtualScene Class

In [ ]:
class VirtualScene:
    """
    Represents a virtual scene containing a pentagon and a list of coins.
    This class handles the visualization and updating of these objects in a 3D environment.

    :cvar LENGTH: Length of the scene (in centimeters).
    :type LENGTH: float
    :cvar WIDTH: Width of the scene (in centimeters).
    :type WIDTH: float
    :ivar pentagon: An instance of the Pentagon class to be visualized in the scene.
    :type pentagon: Pentagon
    :ivar coin_list: A list of Coin objects to be visualized in the scene.
    :type coin_list: list
    :ivar vis: The visualizer object for rendering the scene.
    :type vis: o3d.visualization.Visualizer
    """
    # Scene dimensions (in cm)
    LENGTH = 29.7 
    WIDTH = 21.0  

    def __init__(self, pentagon: Pentagon, coin_list: list) -> None:
        """
        Initializes a virtual scene with a pentagon and a list of coins.

        :param pentagon: An instance of the Pentagon class to be visualized in the scene.
        :type pentagon: Pentagon
        :param coin_list: A list of Coin objects to be visualized in the scene.
        :type coin_list: list

        :rtype: None
        """
        # Assign the pentagon and coins to instance variables
        self.pentagon = pentagon
        self.coin_list = coin_list

        # Visualizer for rendering the 3D objects
        self.vis = o3d.visualization.Visualizer()

        # Set the window position for the visualizer
        left = int((((SCREEN_WIDTH / WINDOW_BASE_WIDTH) / 2.0) + SCREEN_WIDTH / 2.0) - 10)
        top = int((SCREEN_HEIGHT - WINDOW_BASE_HEIGHT) / 2.0)
        self.vis.create_window(
            'Virtual Reality',
            width=WINDOW_BASE_WIDTH,
            height=WINDOW_BASE_HEIGHT,
            left = left, top = top, visible = True)
        
        # Set the background color to black
        self.vis.get_render_option().background_color = np.asarray([0, 0, 0])

        # Create the plane and axes for the scene
        self.create_plane()
        self.create_axes()

        # Add the pentagon to the visualizer if it has vertices
        if len(pentagon.get_triangle_mesh().vertices) > 0:  
            self.vis.add_geometry(pentagon.get_triangle_mesh(), reset_bounding_box = False)

        # Add each coin to the visualizer if they have vertices
        for coin in self.coin_list:
            if len(coin.get_triangle_mesh().vertices) > 0:
                self.vis.add_geometry(coin.get_triangle_mesh(), reset_bounding_box = False)
                coin.is_added_in_visualizer = True
                
        # Register an animation callback to update the coins' positions and rotations
        self.vis.register_animation_callback(self.update_animation)
    
    def run(self) -> None:
        """
        Starts the visualization loop and keeps the window open until the user exits.

        :rtype: None
        """
        self.vis.run()
        self.vis.destroy_window()

    def create_plane(self) -> None:
        """
        Creates and adds a textured plane to the scene to serve as the background or floor.

        :rtype: None
        """
        # Define the vertices of the plane
        vertices = np.array([
            [0, 0, 0],  # Lower-left corner
            [self.LENGTH, 0, 0],   # Lower-right corner
            [self.LENGTH, self.WIDTH, 0],    # Upper-right corner
            [0, self.WIDTH, 0]   # Upper-left corner
        ])

        # Define the two triangles that make up the plane's faces
        faces = [
            [0, 1, 2],  # Bottom triangle
            [0, 2, 3]  # Top triangle
        ]

        # Create a TriangleMesh object for the plane
        plane = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(vertices),
            o3d.utility.Vector3iVector(faces))

        # Define UV texture coordinates for the plane's vertices
        uvs = np.array([
            [0, 0],  # Vertex 0
            [1, 0],  # Vertex 1
            [1, 1],  # Vertex 2
            [0, 1]   # Vertex 3
        ]) # (4, 2)

        # Define the UVs for the faces (order should match the triangle definition)
        triangles_uvs = np.array([
            uvs[0], uvs[1], uvs[2],
            uvs[0], uvs[2], uvs[3]
        ])  # (6, 2)

        # Ensure UV mapping is correct
        assert triangles_uvs.shape == (6, 2)

        # Apply the UV coordinates to the plane
        plane.triangle_uvs = o3d.utility.Vector2dVector(triangles_uvs)

        # Load the texture image for the plane
        texture_image = np.asarray(Image.open("./media/images/texture_plane.png"))
        plane.textures = [o3d.geometry.Image(texture_image)]

        # Assign material IDs to the faces of the plane
        plane.triangle_material_ids = o3d.utility.IntVector([0]*len(faces))

        # Compute vertex normals for realistic lighting/shading effects
        plane.compute_vertex_normals()

        # Add the plane to the visualizer
        self.vis.add_geometry(plane, reset_bounding_box = True)
    
    def create_axes(self) -> None:
        """
        Creates and adds the XYZ axes to the scene for reference.

        :rtype: None
        """
        # Create a coordinate frame to represent the XYZ axes
        world_origin_axes = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size = 3, origin = [0, 0, 0])
        world_origin_axes.compute_vertex_normals()

        # Add the axes to the visualizer
        self.vis.add_geometry(world_origin_axes, reset_bounding_box = False)

    def update_animation(self, visualizer: o3d.visualization.Visualizer) -> None:
        """
        Updates the positions and rotations of the coins in the scene, and re-renders them.
        This method is called during each animation frame to refresh the positions of the 
        pentagon and the coins in the visualizer.

        :param visualizer: The visualizer object responsible for rendering the scene.
        :type visualizer: o3d.visualization.Visualizer

        :rtype: None
        """
        # Return the pentagon to its original position
        self.pentagon.get_triangle_mesh().translate(
            -(self.pentagon.shared_variables.get_pose()), relative=True)
        
        # Set the pentagon's new position
        self.pentagon.shared_variables.set_pose(self.pentagon.shared_variables.get_new_pose()) 
        
        # Move the pentagon to its new position
        self.pentagon.get_triangle_mesh().translate(
            self.pentagon.shared_variables.get_pose(), relative=True)
        
        # Update the pentagon's geometry in the visualizer
        self.vis.update_geometry(self.pentagon.get_triangle_mesh())
        
        # Update the position and rotation for each coin
        for coin in self.coin_list:
            coin.update_rotation()
            coin.update_pose()
            
            # If the coin is visible, update its geometry in the visualizer
            if coin.shared_variables.is_visible():
                if not coin.is_added_in_visualizer:
                    self.vis.add_geometry(coin.get_triangle_mesh(), reset_bounding_box=False)
                    coin.is_added_in_visualizer = True

                self.vis.update_geometry(coin.get_triangle_mesh())
            else:
                # If the coin is not visible, remove it from the visualizer
                self.vis.remove_geometry(coin.get_triangle_mesh(), reset_bounding_box=False)
                coin.is_added_in_visualizer = False

## 7. Definition and Implementation of the GetTheCoins Class

In [ ]:
class GetTheCoins(threading.Thread):
    """
    The GetTheCoins class is responsible for managing the real-world interaction with the camera, 
    processing ArUco marker detections, and updating the virtual environment based on camera 
    pose and object tracking.

    :cvar MARKER_REAL_WIDTH: The real-world width of the ArUco marker in meters.
    :type MARKER_REAL_WIDTH: float
    :cvar MARKER_MARGIN: Margin in meters used for detecting ArUco markers.
    :type MARKER_MARGIN: float
    :cvar MAP_SIZE_X: The width of the map area in meters.
    :type MAP_SIZE_X: float
    :cvar MAP_SIZE_Y: The height of the map area in meters.
    :type MAP_SIZE_Y: float
    :cvar current_id: The current detected marker ID.
    :type current_id: int
    :cvar last_id: The last detected marker ID.
    :type last_id: int
    :cvar p_matrix: The camera projection matrix used for marker detection.
    :type p_matrix: np.array
    :cvar transformation_matrix: The transformation matrix from marker
                                 detection to world coordinates.
    :type transformation_matrix: np.array
    :cvar plane_points: The coordinates of the 3D corners of the map's plane.
    :type plane_points: np.array
    :cvar objpoints: The 3D coordinates of the corners of the ArUco marker.
    :type objpoints: np.array
    :cvar BASE_WIDTH: The base width of the video feed.
    :type BASE_WIDTH: int
    :cvar BASE_HEIGHT: The base height of the video feed.
    :type BASE_HEIGHT: int
    """
    # Define the real width of the ArUco marker (in meters)
    MARKER_REAL_WIDTH = 0.049 
    MARKER_MARGIN = 0.01 

    # Define the map area size (in meters)
    MAP_SIZE_X = 0.297
    MAP_SIZE_Y = 0.210 

    # The current and last detected marker ID, and matrix related to the transformation
    current_id: int
    last_id:int = -1
    p_matrix: np.array
    transformation_matrix: np.array

    # Define the 3D coordinates of the corners of the map's plane
    plane_points = np.array([
        [0.0, 0.0, 0.0],  # Bottom-left corner of the map
        [0.0, MAP_SIZE_Y, 0.0],  # Top-left corner of the map
        [MAP_SIZE_X, MAP_SIZE_Y, 0.0],  # Top-right corner of the map
        [MAP_SIZE_X, 0.0, 0.0]  # Bottom-right corner of the map
    ])

    # Define the coordinates of the 3D corners of the ArUco marker
    objpoints = np.array([
        [-MARKER_REAL_WIDTH / 2, MARKER_REAL_WIDTH / 2, 0],  # Top-left
        [MARKER_REAL_WIDTH / 2, MARKER_REAL_WIDTH / 2, 0],   # Top-right
        [MARKER_REAL_WIDTH / 2, -MARKER_REAL_WIDTH / 2, 0],  # Bottom-right
        [-MARKER_REAL_WIDTH / 2, -MARKER_REAL_WIDTH / 2, 0]  # Bottom-left
    ], dtype=np.float32)

    # The base resolution for the video feed (this may update later if the frame is not this size)
    BASE_WIDTH: int = 640
    BASE_HEIGHT: int = 480

    def __init__(self, pentagon: Pentagon, coin_list: list[Coin]) -> None:
        """
        Initializes the GetTheCoins class with the provided parameters
        and sets up the necessary components, such as camera calibration,
        sound, and virtual game initialization.

        :param pentagon: The virtual pentagon object to track and update in the virtual world.
        :type pentagon: Pentagon

        :param coin_list: List of coins that will be collected in the virtual game.
        :type coin_list: list[Coin]

        :rtype: None
        """
        super().__init__()

        # Load and resize the coin image
        self.coin_img = cv2.imread('./media/images/coin.png', cv2.IMREAD_UNCHANGED)
        self.coin_img = cv2.resize(self.coin_img, (30, 30))

        # Game state initialization
        self.game_on = False

        # Timing variables for game duration
        self.START_TIME = 0.0
        self.current_time = 0.0
        self.MAX_TIME = 60  # Game duration set to 60 seconds
        self.current_time = 0
        self.current_text = ""

        # Initialize pygame for sound handling
        pygame.mixer.init()
        self.coin_sound = pygame.mixer.Sound('./media/audio/coin_sound.mp3')
        
        # Initialize coin collection variables
        self.coins_collected = 0
        self.MIN_COINS = 20  # Minimum number of coins to collect to win

        # Set up initial variables for the pentagon and coin list
        self.pentagon = pentagon
        self.coin_list = coin_list

        # Load camera calibration parameters from file
        camera_calibration_parameters_filename = './media/calibration/intrinsec_parameters.yaml'
        cv_file = cv2.FileStorage(camera_calibration_parameters_filename, cv2.FILE_STORAGE_READ)
        self.camera_matrix = cv_file.getNode('K').mat()
        self.dist_coeffs = cv_file.getNode('D').mat()
        cv_file.release()

        # Open the video capture device (camera)
        self.video = cv2.VideoCapture(2)
        
        # Set the FPS for the video feed
        self.fps = 30

        # Load the predefined ArUco marker dictionary (6x6 markers with 250 possible IDs)
        self.marker_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

        # Define parameters for ArUco marker detection
        self.param_markers = aruco.DetectorParameters()
        self.param_markers.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX

        # Create a new process for handling the virtual scene updates
        self.virtual_scene_process = multiprocessing.Process(
            target = self.run_virtual_scene, args = (self.pentagon, self.coin_list))
    
    def run(self) -> None:
        """
        Main method for running the game loop, displaying the camera feed, and
        processing marker detection.

        This method initializes the game by displaying the start menu and waits
        for the game to begin. Once the game is started, it enters a loop where
        it processes the video feed, detects ArUco markers, estimates their pose,
        and renders the 3D virtual scene with coins, axes, and planes based on
        the detected markers. It also tracks the position of the objects in the
        scene, checks for coin collection, and spawns new coins.

        The loop continues until the user quits the game (by pressing 'q') or the
        video feed ends.

        :rtype: None
        """
        # Start the game menu
        self.start_menu()

        # If the game is active, start processing
        if self.game_on:

            if self.video.isOpened():

                # Start the virtual scene process
                self.virtual_scene_process.start()

                # STEP 1. Create window for displaying the game feed -------------------

                self.game_window_name = 'Get the Coins'
                cv2.namedWindow(self.game_window_name, cv2.WINDOW_NORMAL)
                cv2.resizeWindow(
                    self.game_window_name, WINDOW_BASE_WIDTH, WINDOW_BASE_HEIGHT)

                left = int(((((SCREEN_WIDTH / 2.0) - WINDOW_BASE_WIDTH) / 2.0)) + 10)
                top = int((SCREEN_HEIGHT - WINDOW_BASE_HEIGHT) / 2.0)
                cv2.moveWindow(self.game_window_name, left, top)

                # STEP 2. Definition of control variables for the main game loop -------

                key = -1  # to start key must be different from 'q'
                window_is_visible = True
                delay = 1.0 / self.fps  # FPS delay
                key_wait_delay = 1  # Key press delay (in milliseconds)
                self.START_TIME = time.time()

                # STEP 2. Main game loop -----------------------------------------------

                while ((self.video.isOpened()) and (key != ord('q'))
                        and window_is_visible and self.game_on 
                        and self.virtual_scene_process.is_alive()):

                    ret_ok, frame = self.video.read()

                    # Undistort the captured frame using the camera calibration data
                    frame = cv2.undistort(
                        frame, self.camera_matrix, self.dist_coeffs, None, self.camera_matrix)

                    if (frame is not None) and (ret_ok):
                        
                        frame_height, frame_width, _ = frame.shape
                        
                        # Update the base frame dimensions
                        self.BASE_WIDTH = frame_width
                        self.BASE_HEIGHT = frame_height
                        
                        # Show window images
                        if key != ord('q'):
                        
                            # Convert frame to grayscale for ArUco marker detection
                            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                            # Detect ArUco markers in the grayscale image
                            marker_corners, marker_IDs, _ = aruco.detectMarkers(
                                gray_frame,
                                self.marker_dict,
                                parameters=self.param_markers
                            )

                            if len(marker_corners) > 0:
                                # Estimate the pose of the detected markers
                                rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
                                    marker_corners,
                                    self.MARKER_REAL_WIDTH,
                                    self.camera_matrix, self.dist_coeffs
                                )

                                for i, ids in enumerate(marker_IDs):

                                    if (marker_IDs.size == 1) and (ids[0] == 0):
                                        self.current_id = 0
                                    else:
                                        self.current_id = 1

                                    if self.current_id != self.last_id:
                                        self.transformation_matrix = self.update_transformation_matrix()
                                        self.last_id = self.current_id

                                    if (self.current_id == ids[0]):

                                        # Auto-track the object in the scene
                                        bbox = self.auto_tracking(frame)

                                        # Update the projection matrix with the marker's pose
                                        self.p_matrix = self.get_projection_matrix(
                                            self.camera_matrix, rvecs[i], tvecs[i])
                                        
                                        corners = marker_corners[i].astype(np.int32)

                                        # Draw polygon around the detected marker
                                        cv2.polylines(
                                            frame, [corners], True,
                                            (145, 89, 222), 2, cv2.LINE_AA)

                                        # Draw the virtual plane, coins, and axes in the scene
                                        self.draw_plane(frame)
                                        self.draw_coins(frame)
                                        self.draw_axes(frame)
                                        
                                        """
                                        # Draw the axes of the detected markers (representing 3D pose)
                                        cv2.drawFrameAxes(
                                            frame, self.camera_matrix,
                                            self.dist_coeffs, rvecs[i], tvecs[i], 0.02, 2)
                                        """

                                        # Display marker ID on the screen
                                        corners = corners.reshape(4, 2)
                                        corners = corners.astype(int)
                                        top_right = corners[0].ravel()
                                        offset_x = 10  # X offset for the text
                                        offset_y = 10  # Y offset for the text
                                        adjusted_position = (
                                            int(top_right[0]) + offset_x,
                                            int(top_right[1]) - offset_y)
                                        
                                        # Display the marker ID on the frame
                                        cv2.putText(
                                            frame,
                                            f"ID: {int(ids[0])}",
                                            adjusted_position,
                                            cv2.FONT_HERSHEY_PLAIN,
                                            1.2,
                                            (145, 89, 222), 2, cv2.LINE_AA
                                        )

                                        # If a bounding box is found, draw it and calculate centroid
                                        if bbox is not None:
                                            p1 = (int(bbox[0]), int(bbox[1]))
                                            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                                            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
                                            
                                            # Calculate centroid of the bounding box
                                            centroid_x = int((p1[0] + p2[0]) / 2)
                                            centroid_y = int((p1[1] + p2[1]) / 2)
                                            centroid = (centroid_x, centroid_y)

                                            # Draw the centroid point
                                            cv2.circle(frame, centroid, 3, (0, 255, 0), -1)  # Green

                                            # Update the virtual position of the pentagon based on centroid position
                                            pixel_coords = np.array([[centroid[0], centroid[1]]])  # (1, 2)
                                            new_pose = self.pixel_to_3d(pixel_coords)
                                            self.pentagon.update_virtual_pose(new_pose)

                                            # Check for coin collection and spawn new coins
                                            self.check_coin_collected()
                                            self.spawn_new_coins()
                                                
                                        else:
                                            logger.warning("No object detected, retrying in next frame.")

                                        """
                                        # Proyectar los puntos 3D a la imagen 2D
                                        U, _ = cv2.projectPoints(
                                            self.objpoints, rvecs[0], tvecs[0],
                                            self.camera_matrix, self.dist_coeffs)

                                        # Calcular el error de retroproyección
                                        error = np.asarray(U[0,0]-self.imgpoints[0][:,0,:])
                                        error = np.linalg.norm(error)/len(error)
                                        logger.info("Retroprojection error with OpenCV for first image:", error)
                                        """

                            # Update the game interface
                            self.update_interface(frame)
                            
                            # Resize the frame and display
                            resized_frame = cv2.resize(
                                frame, (self.BASE_WIDTH * 2, self.BASE_HEIGHT * 2))
                            
                            cv2.imshow(self.game_window_name, resized_frame)

                        # Wait for key input (q to quit)
                        key = cv2.waitKey(key_wait_delay)

                        # Check if the window is still open
                        if cv2.getWindowProperty(
                            self.game_window_name, cv2.WND_PROP_VISIBLE) == 0:
                            window_is_visible = False
                
                # Clean up when the video feed ends
                cv2.destroyAllWindows()
                self.virtual_scene_process.kill()
                
                # End the game
                self.end_game()

    #-------------------------------------------------------------------------#
    # CLICK EVENT AND START MENU    
    #-------------------------------------------------------------------------#
    
    def click_event(self, event: int, x: int, y: int, flags: int, param: any) -> None:
        """
        Handles mouse click events to start the game when the "START" button is clicked.

        This method is triggered by a mouse click event. It checks if the click was
        inside the "START" button area and, if so, sets the game to start.

        :param event: The event type (e.g., left mouse button click).
        :type event: int
        :param x: The x-coordinate of the mouse click in the window.
        :type x: int
        :param y: The y-coordinate of the mouse click in the window.
        :type y: int
        :param flags: Any relevant flags associated with the event (not used here).
        :type flags: int
        :param param: Any additional parameter (not used here).
        :type param: any

        :rtype: None
        """
        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if the click is within the "START" button area
            if (self.button_x <= x <= self.button_x + self.button_w 
                and self.button_y <= y <= self.button_y + self.button_h):
                
                # If clicked within the button, set game to start
                self.game_on = True
           
    def show_start_window(self) -> None:
        """
        Displays the start menu with a welcome message and the "START" button.

        This method renders the start menu with the game's welcome message and
        a "START" button. It centers the message vertically and horizontally on
        the screen. The "START" button is drawn and is ready for user interaction
        to begin the game.

        :rtype: None
        """
        # Set background color for the start menu
        self.window[:] = 50
        
        # Split the message into lines
        lines = self.message.split('\n')
        text_height = len(lines) * 40
        
        # Calculate the initial Y position to center the message vertically
        y0 = (WINDOW_BASE_HEIGHT - text_height) // 2 + 20
        
        # Loop through each line of the message and render it
        for i, line in enumerate(lines):
            text_size, _ = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            text_x = (WINDOW_BASE_WIDTH - text_size[0]) // 2
            text_y = y0 + i * 40
            cv2.putText(self.window, line, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Draw the "START" button
        cv2.rectangle(self.window, (self.button_x, self.button_y),
                    (self.button_x + self.button_w, self.button_y + self.button_h), 
                    (0, 200, 0), -1)

        # Render the "START" button text in the center
        button_text = "START"
        button_text_size, _ = cv2.getTextSize(button_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        button_text_x = self.button_x + (self.button_w - button_text_size[0]) // 2
        button_text_y = self.button_y + (self.button_h + button_text_size[1]) // 2
        
        cv2.putText(
            self.window, button_text, (button_text_x, button_text_y),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # Display the start menu window
        cv2.imshow(self.start_window_name, self.window)

    def start_menu(self) -> None:
        """
        Initializes and displays the start menu window with a welcome message.
        Waits for the user to click "START" to begin the game.

        This method creates a window with a welcome message and a button. It
        waits for the user to click the "START" button to begin the game. The
        window is also set up to close if the user presses 'q'.

        :rtype: None
        """
         # Define the welcome message with game instructions
        self.message =  "Welcome to 'Get the Coins'!\n\n" \
                        "In this game, you have to collect the maximum\n" \
                        "amount of coins possible in 1 min.\n\n" \
                        "MINIMUM AMOUNT OF COINS TO WIN: 20\n\n" \
                        "If you are ready, press START.\n\n"

        # Create a blank window to display the start menu
        self.window = np.zeros(
            (WINDOW_BASE_HEIGHT, WINDOW_BASE_WIDTH, 3), dtype=np.uint8) + 50
        
        # Set button dimensions
        self.button_w, self.button_h = 200, 60

        # Set initial button position (centered horizontally)
        self.button_x, self.button_y = int(
            (WINDOW_BASE_WIDTH / 2) - (self.button_w / 2)), 576
        
        # Window name for the start menu
        self.start_window_name = "Welcome to 'Get the Coins'"
        
        # Create a named window for OpenCV
        cv2.namedWindow(self.start_window_name, cv2.WINDOW_NORMAL)

        # Set the initial window size
        cv2.resizeWindow(self.start_window_name, WINDOW_BASE_WIDTH, WINDOW_BASE_HEIGHT)

        # Position the window on the screen
        left = int((((SCREEN_WIDTH) - WINDOW_BASE_WIDTH) / 2.0))
        top = int((SCREEN_HEIGHT - WINDOW_BASE_HEIGHT) / 2.0)
        cv2.moveWindow(self.start_window_name, left, top)

        # Set mouse callback to handle click events
        cv2.setMouseCallback(self.start_window_name, self.click_event)
        
        # Keep displaying the start window until the game starts
        while not self.game_on:
            
            self.show_start_window()
            
            # Close window if 'q' is pressed
            if cv2.waitKey(1) == ord('q'):
                self.game_on = False
                break
        
        # Close the start menu window after the game starts
        cv2.destroyWindow(self.start_window_name)

    @staticmethod
    def run_virtual_scene(pentagon: Pentagon, coin_list: list[Coin]) -> None:
        """
        Initializes and runs the virtual scene with the given pentagon and coins.

        This static method creates a new 'VirtualScene' instance with the provided
        pentagon and coin list, and then starts the scene's execution.

        :param pentagon: The pentagon object that will interact with the scene.
        :type pentagon: Pentagon  # Replace with actual type if available

        :param coin_list: A list of coin objects to be included in the scene.
        :type coin_list: list  # List of Coin objects or similar
        
        :rtype: None
        """
        virtual_scene = VirtualScene(pentagon, coin_list)
        virtual_scene.run()
    
    #-------------------------------------------------------------------------#
    # RENDER AND TRANSFORMATIONS
    #-------------------------------------------------------------------------#

    def draw_plane(self, frame: np.array) -> None:
        """
        Draws a plane on the 3D scene and projects it into the 2D image plane.
        
        This method takes the plane points, applies the transformation matrix,
        and projects the transformed points onto the 2D frame using a
        perspective transformation, drawing the resulting polygon on the frame.

        :param frame: The current video frame to draw the plane onto.
        :type frame: np.array
        
        :rtype: None
        """
        # Convert points to homogeneous coordinates (add column of ones)
        points_homogeneous = np.hstack(
            (self.plane_points, np.ones((self.plane_points.shape[0], 1))))

        # Apply transformation matrix to all points at once
        transformed_points = np.dot(
            self.transformation_matrix, points_homogeneous.T).T

        # Convert back from homogeneous coordinates
        transformed_points = transformed_points[:, :3]

        # Use perspective transform to get pixel coordinates
        dst = cv2.perspectiveTransform(
            transformed_points.reshape(-1, 1, 3), self.p_matrix)
        
        pixel_points = np.int32(dst).reshape(-1, 2)  # Ensure format is correct

        # Define color for drawing
        if self.current_id == 0:
            color_2 = (255, 150, 70)  # Set color if it's the first ID
        else:
            color_2 = (70, 150, 255)  # Set color if it's the second ID

        # Draw the plane as a polygon
        map_corners = np.array([pixel_points])      
        cv2.polylines(frame, [map_corners], True, color_2, 2, cv2.LINE_AA)
 
    def draw_coins(self, frame: np.array) -> None:
        """
        Renders coins in the current frame, applying
        transformations and drawing them.
        
        This method iterates over the list of coins, applying their
        respective transformations (rotation, translation), and then
        projects them onto the 2D image frame for rendering.

        :param frame: The current video frame where the coins will be drawn.
        :type frame: np.array
        
        :rtype: None
        """
        DEFAULT_COLOR = (118, 211, 255)  # Default color for coins (in BGR)

        for coin in self.coin_list:

            if coin.shared_variables.is_visible():
                # Get vertices and other properties of the coin
                vertices = coin.augmented_reality_obj.vertices

                # Translation in X (in meters)
                dx = coin.shared_variables.get_pose()[0] / 100

                # Translation in Y (in meters)
                dy = coin.shared_variables.get_pose()[1] / 100

                # Create rotation matrix for rotation around the Z axis
                angle = coin.shared_variables.get_rotation()  # Rotation in radians
                rotation_matrix = np.array([
                    [np.cos(angle), -np.sin(angle), 0],
                    [np.sin(angle), np.cos(angle), 0],
                    [0, 0, 1]
                ])

                # Process each face of the coin
                for face in coin.augmented_reality_obj.faces:
                    face_vertices = face[0]
                    points = np.array([vertices[vertex - 1] for vertex in face_vertices])

                    # Apply the rotation to the points
                    points = np.dot(points, rotation_matrix.T)  # Apply rotation matrix

                    # Apply translation
                    points[:, 0] += dx  # Apply translation in X
                    points[:, 1] += dy  # Apply translation in Y

                    # Convert to homogeneous coordinates
                    points_homogeneous = np.hstack(
                        (points, np.ones((points.shape[0], 1))))  # Add column of ones

                    # Apply the transformation matrix to get the new coordinates
                    transformed_points = np.dot(
                        self.transformation_matrix, points_homogeneous.T).T 

                    # Convert to 3D coordinates
                    transformed_points = transformed_points[:, :3]

                    # Get the 2D pixel coordinates from the 3D transformed points
                    dst = cv2.perspectiveTransform(
                        transformed_points.reshape(-1, 1, 3), self.p_matrix)
                    imgpts = np.int32(dst)

                    # Fill the convex polygon (coin) in the frame
                    cv2.fillConvexPoly(frame, imgpts, DEFAULT_COLOR)
    
    def draw_axes(self, frame: np.array, length: float = 0.05) -> None:
        """
        Draws the 3D axes (X, Y, Z) on the given image frame.

        This method computes the transformation of the 3D axes (X, Y, Z) based
        on the transformation matrix and projects them onto the 2D plane of the
        image using the camera's projection matrix.

        :param frame: The image frame where the axes will be drawn.
        :type frame: np.array
        :param length: The length of the axes. Default is 0.05.
        :type length: float

        :rtype: None
        """
        # Define the points for the axes (origin and axis endpoints)
        origin = np.array([0, 0, 0])
        x_axis = np.array([length, 0, 0])
        y_axis = np.array([0, length, 0])
        z_axis = np.array([0, 0, length])

        # Create an array of points including the origin and axis endpoints
        points = np.array([origin, x_axis, y_axis, z_axis])

        # Convert points to homogeneous coordinates (add a column of ones)
        points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1)))) 

        # Multiply the transformation matrix by all points at once
        transformed_points = np.dot(
            self.transformation_matrix, points_homogeneous.T).T 

        # Convert back to 3D coordinates (remove the homogeneous component)
        transformed_points = transformed_points[:, :3]

        # Get the pixel coordinates using cv2.perspectiveTransform
        dst = cv2.perspectiveTransform(
            transformed_points.reshape(-1, 1, 3), self.p_matrix)
        
        pixel_points = np.int32(dst).reshape(-1, 2)  # Ensure the correct format

        # Draw the axes
        # X axis in red
        cv2.line(
            frame, tuple(pixel_points[0]),
            tuple(pixel_points[1]), (0, 0, 255), 5)
        
        # Y axis in green
        cv2.line(
            frame, tuple(pixel_points[0]),
            tuple(pixel_points[2]), (0, 255, 0), 5)
        
        # Z axis in blue
        cv2.line(
            frame, tuple(pixel_points[0]),
            tuple(pixel_points[3]), (255, 0, 0), 5)

    def get_projection_matrix(self, mtx: np.array, rvec: np.array, tvec: np.array) -> np.array:
        """
        Calculates the 3D to 2D projection matrix using the camera parameters.

        This method uses the camera intrinsic matrix, rotation vector, and
        translation vector to compute the 3D to 2D projection matrix, which
        can be used to project 3D world coordinates onto a 2D image plane.

        :param mtx: Camera intrinsic matrix (calibration matrix).
        :type mtx: np.array
        :param rvec: Rotation vector (camera orientation).
        :type rvec: np.array
        :param tvec: Translation vector (camera position).
        :type tvec: np.array

        :return: 3D to 2D projection matrix.
        :rtype: np.array
        """
        # Camera intrinsic matrix
        K = mtx

        # Convert the rotation vector to a rotation matrix using cv2.Rodrigues
        R = cv2.Rodrigues(rvec)[0]

        # Translation vectors
        T = tvec.T

        # Concatenate the rotation and translation matrices
        RT = np.concatenate((R, T), axis=1)
        
        # Calculate the final projection matrix
        Pest0 = np.dot(K, RT)

        return Pest0
    
    def update_transformation_matrix(self) -> np.array:
        """
        Updates the transformation matrix with translation and rotation values.

        This method calculates the translation and rotation matrices based on
        the current marker ID, applies the necessary translation and rotation,
        and returns the updated transformation matrix.

        :return: Updated transformation matrix.
        :rtype: np.array
        """
        # Define the translation differences depending on the case
        x_diff: float
        y_diff: float
        angle: float

        if self.current_id == 0:
            # If the marker is the first, apply negative translation
            x_diff = -(self.MARKER_MARGIN + (self.MARKER_REAL_WIDTH / 2.0))
            y_diff = -(self.MARKER_MARGIN + (self.MARKER_REAL_WIDTH / 2.0))
            angle = 0.0
        else:
            # If the marker is the second one, apply positive translation
            x_diff = self.MAP_SIZE_X - (
                self.MARKER_MARGIN + (self.MARKER_REAL_WIDTH / 2.0))
            
            y_diff = self.MAP_SIZE_Y - (
                self.MARKER_MARGIN + (self.MARKER_REAL_WIDTH / 2.0))
            
            angle = 180.0

        # Calculate cosine and sine for the rotation angle (+ 180)
        cos = math.cos(math.radians(angle))
        sin = math.sin(math.radians(angle))

        # Translation matrix
        T_translation = np.array([
            [1.0, 0.0, 0.0, x_diff],
            [0.0, 1.0, 0.0, y_diff],
            [0.0, 0.0, 1.0,    0.0],
            [0.0, 0.0, 0.0,    1.0]
        ])

        # Rotation matrix
        T_rotation = np.array([
            [cos, -sin, 0.0, 0.0],
            [sin,  cos, 0.0, 0.0],
            [0.0,  0.0, 1.0, 0.0],
            [0.0,  0.0, 0.0, 1.0]
        ])

        # Multiply the translation and rotation matrices
        # to get the final transformation matrix
        T = T_translation @ T_rotation

        return T

    def pixel_to_3d(self, pixel_coords: np.array) -> np.array:
        """
        Converts 2D pixel coordinates to 3D world coordinates using a
        projection matrix and transformation matrix.

        This method first converts the 2D pixel coordinates to homogeneous
        coordinates, applies a correction to the Z-axis, and then uses the
        projection matrix to calculate the 3D coordinates. The resulting 
        coordinates are then transformed using the transformation matrix
        and returned in Cartesian form.

        :param pixel_coords: 2D coordinates in the image (pixel coordinates).
        :type pixel_coords: ndarray

        :return: 3D coordinates in the world, transformed using the transformation matrix.
        :rtype: np.array
        """
        # Convert the 2D pixel coordinates to homogeneous
        # coordinates by adding a column of ones
        pixel_homogeneous = np.hstack(
            (pixel_coords, np.ones((pixel_coords.shape[0], 1)))).reshape(3, 1)

        # Apply a correction to the Z-axis height of the object
        #  which adds a small translation in the Z-axis

        # Translation of hafl pentagon height along the Z-axis
        z_axis = (PentagonBase.PENTAGON_HEIGHT * 0.01 / 2)

        translation_z = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, z_axis],
            [0, 0, 0, 1]
        ])

        # Apply the translation to the 3D homography matrix
        homography_3d = np.dot(self.p_matrix, translation_z)
        homography = homography_3d[:, [0, 1, 3]]
        homography_inv = np.linalg.inv(homography)

        # Multiply the homogeneous coordinates with the inverse
        # homography to get the 3D coordinates in homogeneous form
        coords_2d = np.dot(homography_inv, pixel_homogeneous)

        # Normalize the 3D coordinates by dividing by the third 
        # component (z) to get Cartesian coordinates
        coords_2d[0] = coords_2d[0] / coords_2d[2]  # Normalize X
        coords_2d[1] = coords_2d[1] / coords_2d[2]  # Normalize Y
        coords_2d[2] = coords_2d[2] / coords_2d[2]  # Normalize Z

        # Convert back to homogeneous coordinates by adding an additional row of ones
        coords_2d_homogeneous = np.vstack(
            (coords_2d, np.ones((1, coords_2d.shape[1])))) 

        # Set the Z component to 0 to project onto the 2D plane
        coords_2d_homogeneous[2] = 0.0

        # Copy the transformation matrix to apply further modifications
        transformation_matrix_copy = self.transformation_matrix.copy()
        if self.current_id == 0:
            transformation_matrix_copy[:-2, -1] *= -1

        # Apply the transformation matrix to the 3D coordinates
        transformed_coords_2d = np.dot(
            transformation_matrix_copy, coords_2d_homogeneous)

        # Apply the transformation to the coordinates
        x_3d = transformed_coords_2d[0, :]  # Transformed X coordinates
        y_3d = transformed_coords_2d[1, :]  # Transformed Y coordinates
        z_3d = np.zeros_like(x_3d)          # Transformed Z coordinates

        # Flatten the coordinates for output
        new_pose_transformed = np.array([x_3d, y_3d, z_3d]).flatten()

        # Return the transformed 3D coordinates
        return new_pose_transformed

    def auto_tracking(self, frame: np.array) -> tuple:
        """
        Performs automatic object tracking based on color.

        This method detects objects within a specified green color range in the
        given frame. It applies noise reduction using morphological operations,
        finds contours, and filters them based on size and aspect ratio to find the 
        best bounding box for the object.

        :param frame: The current image frame to process.
        :type frame: ndarray

        :return: The bounding box of the detected object, or None if no valid object
                 is detected.
        :rtype: tuple or None
        """
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                
        # Define the green color range in HSV
        lower_green = np.array([35, 50, 200])  # Minimum green value
        upper_green = np.array([80, 255, 255])  # Maximum green value
        
        # Create a mask to filter pixels within the green color range
        mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # Filter noise using morphological operations
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        best_bbox = None
        
        # Filter contours by size and aspect ratio to avoid noise
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            area = w * h
            aspect_ratio = w / float(h)
            
            # Filter by size and aspect ratio to avoid incorrect detections
            if 300 < area < 8000 and 0.5 < aspect_ratio < 1.5:  
                best_bbox = (x, y, w, h)
        
        return best_bbox
    
    def check_coin_collected(self) -> None:
        """
        Checks if the pentagon has collected any coins by checking for collisions
        between the pentagon and each coin in the coin_list.

        This method calculates the Euclidean distance between the pentagon and
        each visible coin. If a collision is detected, the coin is hidden, the
        collection time is recorded, and the coin count is incremented.

        :rtype: None
        """
        # Check if game time is still running
        if self.current_time < self.MAX_TIME:
            # Get position of the pentagon
            pentagon_position = self.pentagon.shared_variables.get_pose()

            # Get radius of the pentagon
            pentagon_radius = self.pentagon.shared_variables.get_pentagon_radius()

            # Check for collisions through the coin list
            for coin in self.coin_list:
                if coin.shared_variables.is_visible():  # Only check for visible coins
                    coin_position = coin.shared_variables.get_pose()  # Get position of the coin
                    coin_radius = coin.shared_variables.get_coin_radius()  # Get radius of the coin

                    # Calculate the Euclidean distance between the pentagon and the coin
                    distance = ((pentagon_position[0] - coin_position[0]) ** 2 + 
                                (pentagon_position[1] - coin_position[1]) ** 2) ** 0.5

                    # If the distance is less than the sum of the radii, it's a collision
                    if distance < (pentagon_radius + coin_radius):
                        # Hide the coin
                        coin.shared_variables.set_visibility(False)

                        # Record the time it disappeared
                        coin.shared_variables.set_last_apparition_time(time.time())

                        # Increment collected coin count
                        self.coins_collected += 1
                        #  Play sound when a coin is collected
                        self.coin_sound.play()

    def spawn_new_coins(self) -> None:
        """
        Respawns coins that are no longer visible after a brief delay.

        This method checks all coins in the list and makes any coin that is no
        longer visible reappear after a delay of 0.5 seconds. The coin's position
        is updated to a random location in the scene.

        :rtype: None
        """
        for coin in self.coin_list:  # Check all coins in the list

            if not coin.shared_variables.is_visible():  # If the coin is not visible
                # Calculate the time since the coin last disappeared
                diff_time = time.time() - coin.shared_variables.get_last_apparition_time()

                if (diff_time) > 0.5:  # If more than 0.5 seconds have passed
                    coin.shared_variables.set_visibility(True)  # Make the coin visible again
                    # Update the coin's position to a random new location
                    coin.update_virtual_pose(
                        CoinBase.random_coin_position(VirtualScene.LENGTH, VirtualScene.WIDTH))

    def update_interface(self, frame: np.array) -> None:
        """
        Updates the on-screen interface, including the timer and the coin counter.

        This method overlays the current time and coin count on the given frame.
        It also handles the display of a timer, coin image, and the coin count.
        If the game time has ended, it shows a "TIME IS UP" message and ends the game.

        :param frame: The current frame from the video stream to overlay information.
        :type frame: ndarray

        :rtype: None
        """
        frame_height, frame_width = frame.shape[:2]
        self.current_time = time.time() - self.START_TIME # Calculate elapsed time

        # If the game time hasn't exceeded the maximum time
        if (self.current_time < self.MAX_TIME):
            self.current_text = f"{self.current_time:.2f}s"  # Format the current time text
            
            # Draw the timer background rectangle
            rect_width, rect_height = 160, 50
            rect_x = frame_width - rect_width - 20
            rect_y = 20
            overlay = frame.copy()

            cv2.rectangle(
                overlay, (rect_x, rect_y), 
                (rect_x + rect_width, rect_y + rect_height), (0, 0, 0), -1)
            
            alpha = 0.5
            cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

            cv2.rectangle(
                frame, (rect_x, rect_y), 
                (rect_x + rect_width, rect_y + rect_height), (0, 153, 153), 2)

            text_size = cv2.getTextSize(
                self.current_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
            
            text_x = rect_x + (rect_width - text_size[0]) // 2
            text_y = rect_y + (rect_height + text_size[1]) // 2

            cv2.putText(
                frame, self.current_text, (text_x, text_y),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 153, 153), 2, cv2.LINE_AA)
            
            # Draw the coin image and counter on the interface
            coin_x, coin_y = 20, 20
            if self.coin_img.shape[2] == 4:  # If the coin image has an alpha channel
                alpha_coin = self.coin_img[:, :, 3] / 255.0
                background_area = frame[coin_y:coin_y + 30, coin_x:coin_x + 30]
                # Blend the coin image with the background area using the alpha channel
                for c in range(3):
                    background_area[:, :, c] = (
                        alpha_coin * self.coin_img[:, :, c] +
                        (1 - alpha_coin) * background_area[:, :, c])
                
                frame[coin_y:coin_y + 30, coin_x:coin_x + 30] = background_area
            else:
                # Draw coin image without alpha blending
                frame[coin_y:coin_y + 30, coin_x:coin_x + 30] = self.coin_img 

            # Display the collected coin count next to the coin image
            coins_text = f"x {self.coins_collected}"
            cv2.putText(
                frame, coins_text, (coin_x + 40, coin_y + 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            
        # If time is over and there's a short delay
        elif (self.current_time > self.MAX_TIME + 2.0):
            self.game_on = False  # End the game

        else:
            # Display "TIME IS UP" text if the game time has passed
            cv2.putText(
                frame, "TIME IS UP",
                (int((frame_width // 2) * 0.2), int((frame_height // 2) * 1.13)), 
                cv2.FONT_HERSHEY_SIMPLEX,
                3, (0, 0, 255), 4, cv2.LINE_AA)
            
    def end_game(self) -> None:
        """
        Ends the game, displays the final results, and closes the game window.

        This function checks if the player has collected enough coins to win,
        displays the final score, and shows a window with the results. After
        the results are shown, it waits for a key press to close the window
        and terminates the game.

        :rtype: None
        """
        coins_color = (0, 0, 255)  # Default color for less than minimum coins collected

        if (self.coins_collected >= self.MIN_COINS):  # Check if enough coins were collected
            coins_color = (0, 255, 0)  # Green if the player wins
        
        # Create a black window for displaying the final score
        end_window = np.zeros((WINDOW_BASE_HEIGHT, WINDOW_BASE_WIDTH, 3), dtype=np.uint8) + 50

        text_prefix = "RECOLLECTED COINS: "
        coins_text = str(self.coins_collected)
        # Calculate the width of the text to center it in the window
        prefix_size = cv2.getTextSize(text_prefix, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)[0]
        monedas_size = cv2.getTextSize(coins_text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)[0]
        ancho_total = prefix_size[0] + monedas_size[0]

        text_x0 = (end_window.shape[1] - ancho_total) // 2
        text_y0 = (end_window.shape[0] // 2) + (prefix_size[1] // 2)

        # Display the collected coins and final score
        cv2.putText(
            end_window, text_prefix, (text_x0, text_y0),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(
            end_window, coins_text, (text_x0 + prefix_size[0], text_y0),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, coins_color, 2, cv2.LINE_AA)
        
        self.end_window_name = "End Results"
        
        # Open a new window to show the results
        cv2.namedWindow(self.end_window_name, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(self.end_window_name, WINDOW_BASE_WIDTH, WINDOW_BASE_HEIGHT)

        # Center the window on the screen
        left = int((((SCREEN_WIDTH) - WINDOW_BASE_WIDTH) / 2.0))
        top = int((SCREEN_HEIGHT - WINDOW_BASE_HEIGHT) / 2.0)
        cv2.moveWindow(self.end_window_name, left, top)

        # Show the final results
        cv2.imshow(self.end_window_name, end_window)
        
        # Wait for a key press to close the window
        key = cv2.waitKey(0)
        cv2.destroyAllWindows()

        pygame.mixer.quit() # Close pygame mixer when the game ends

## 8. Definition and Implementation of the CustomManager Class

In [ ]:
class CustomManager(BaseManager):
    """
    A custom manager class that inherits from 'BaseManager'.

    This class does not modify or add any functionality to the base manager.
    It can serve as a placeholder for custom registration logic or other
    modifications if needed in the future.
    """
    pass

# Register the CoinBase class in the Custom Manager.
CustomManager.register('CoinBase', CoinBase)

# Register the PentagonBase class in the Custom Manager.
CustomManager.register('PentagonBase', PentagonBase)

# Create an instance of the CustomManager
custom_manager = CustomManager()

# Start the custom manager to initiate the process of managing registered classes.
custom_manager.start()

## 9. Object Creation

In [ ]:
# Generate random positions for 5 coins
num_coins = 5

coin_list = []  # List to store the created coin objects

for _ in range(num_coins):
    # Generate a random position for the coin in the virtual scene
    world_pose = CoinBase.random_coin_position(VirtualScene.LENGTH, VirtualScene.WIDTH)
    
    # Create a shared instance of CoinBase with the random position
    shared_coin = custom_manager.CoinBase(world_pose)

    # Create a Coin object using the shared instance
    coin = Coin(shared_coin)
    
    # Append the coin to the coin_list
    coin_list.append(coin)

# Create a shared instance of PentagonBase
shared_pentagon = custom_manager.PentagonBase()

# Create a Pentagon object using the shared instance
pentagon = Pentagon(shared_pentagon)

## 10. Run Game

Initializes and runs the coin collection game with a given pentagon shape and coin list.

In [ ]:
# Create an instance of the game
game = GetTheCoins(pentagon, coin_list)

# Start the game
game.start()

# Wait for the game to finish
game.join()